In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [3]:
!pip install pyarabic
!pip install arabic-stopwords
!pip install qalsadi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from sklearn import metrics
import pandas as pd
from torch import nn
from pyarabic.araby import tokenize
import numpy as np
import pickle
import spacy
import torch

# from model_building import Classifier 
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector
from gru_model import ArabicDataset, Classifier, evaluate, train

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
# needed functions
def print_report(y_pred, y_test):
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [6]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

In [7]:
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))
test_data['category'] = test_data['category'].astype('category').cat.codes

In [8]:
# load word2index dictionary
with open('./vocab/category/word2index.pickle', 'rb') as f:
    word2index = pickle.load(f)

In [9]:
weights_train_matrix = []
for word in word2index:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

td-idf feature

In [17]:
# load SVC model
with open('./models/category/LinearSVC_tfidf.sav', 'rb') as f:
    SVC_model = pickle.load(f)

with open('./models/category/BoWVectorizer.sav', 'rb') as f:
    word_vectorizer = pickle.load(f)

X_test_BoW = word_vectorizer.transform(test_data['text'])
y_pred = SVC_model.predict_proba(test_data['text'])

In [19]:
with open('./models/category/RandomForest_Bow.sav', 'rb') as f:
    rf = pickle.load(f)

y_pred += rf.predict_proba(X_test_BoW)

In [20]:
#load GRU model
# with open('./models/category/GRU_Ara2Vec.pth', 'rb') as f:
#     gru_model = Classifier(weights_train_matrix, n_classes=10, n_layer=2)
#     gru_model.load_state_dict(torch.load(f), strict=False)
#     gru_model.eval()

# test_data_tokenized = test_data['text'].apply(tokenize)
# test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
# test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['category'], word2index['<فراغ>'])
with open('./probability/GRU_prob.pickle', 'rb') as f:
    gru_prob = pickle.load(f)
y_pred += gru_prob

In [21]:
# load transformer probas
# with open('./probability/y_prob_cat.pickle', 'rb') as f:
#     y_prob_category = pickle.load(f)

In [22]:
# y_pred += y_prob_category

In [23]:
y_pred = np.argmax(y_pred, axis=1)
print_report(y_pred, test_data['category'])

              precision    recall  f1-score   support

           0       0.10      0.20      0.13        10
           1       0.84      0.84      0.84       145
           2       0.76      0.66      0.70       545
           3       0.38      0.29      0.33        17
           4       0.42      0.63      0.50       128
           5       0.40      0.02      0.05        82
           6       0.10      0.20      0.13        20
           7       0.20      0.50      0.29         2
           8       0.00      0.00      0.00        15
           9       0.23      0.61      0.33        36

    accuracy                           0.60      1000
   macro avg       0.34      0.40      0.33      1000
weighted avg       0.64      0.60      0.60      1000

accuracy: 0.598


In [16]:
# print_report(np.argmax(y_prob_category, axis=1), test_data['category'])